In [1]:
import os
import sys
import math
import psutil
import sklearn
import humanize
import warnings
import subprocess
import numpy as np
import pandas as pd
import GPUtil as GPU
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.python.client import device_lib


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
import keras
import keras.backend as K
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.initializers import glorot_uniform
from keras.layers.core import Permute, Reshape, RepeatVector
from keras.losses import cosine_proximity, categorical_crossentropy
from keras.layers import Input, Dense, Dropout, CuDNNGRU, Embedding, concatenate, Lambda, multiply, merge, Flatten

Using TensorFlow backend.


In [4]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""    
    def __init__(self, data, sep='\t', session_key='SessionId', item_key='ItemId', time_key='Time', n_samples=-1, itemmap=None, time_sort=False):
        """
        Args:
            path: path of the csv file
            sep: separator for the csv
            session_key, item_key, time_key: name of the fields corresponding to the sessions, items, time
            n_samples: the number of samples to use. If -1, use the whole dataset.
            itemmap: mapping between item IDs and item indices
            time_sort: whether to sort the sessions by time or not
        """
        self.df = data
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key
        self.time_sort = time_sort
        self.add_item_indices(itemmap=itemmap)
        self.df.sort_values([session_key, time_key], inplace=True)

        #Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        #clicks within a session are next to each other, where the clicks within a session are time-ordered.

        self.click_offsets = self.get_click_offsets()
        self.session_idx_arr = self.order_session_idx()
        
    def get_click_offsets(self):
        """
        Return the offsets of the beginning clicks of each session IDs,
        where the offset is calculated against the first click of the first session ID.
        """
        offsets = np.zeros(self.df[self.session_key].nunique() + 1, dtype=np.int32)
        # group & sort the df by session_key and get the offset values
        offsets[1:] = self.df.groupby(self.session_key).size().cumsum()

        return offsets

    def order_session_idx(self):
        """ Order the session indices """
        if self.time_sort:
            # starting time for each sessions, sorted by session IDs
            sessions_start_time = self.df.groupby(self.session_key)[self.time_key].min().values
            # order the session indices by session starting times
            session_idx_arr = np.argsort(sessions_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.session_key].nunique())

        return session_idx_arr
    
    def add_item_indices(self, itemmap=None):
        """ 
        Add item index column named "item_idx" to the df
        Args:
            itemmap (pd.DataFrame): mapping between the item Ids and indices
        """
        if itemmap is None:
            item_ids = self.df[self.item_key].unique()  # unique item ids
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            itemmap = pd.DataFrame({self.item_key:item_ids,
                                   'item_idx':item2idx[item_ids].values})
        
        self.itemmap = itemmap
        self.df = pd.merge(self.df, self.itemmap, on=self.item_key, how='inner')
        
    @property    
    def items(self):
        return self.itemmap.ItemId.unique()
        

In [5]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""    
    def __init__(self, dataset, batch_size=50):
        """
        A class for creating session-parallel mini-batches.
        Args:
             dataset (SessionDataset): the session dataset to generate the batches from
             batch_size (int): size of the batch
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.done_sessions_counter = 0
        
    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        # initializations
        df = self.dataset.df
        session_key='SessionId'
        item_key='ItemId'
        time_key='TimeStamp'
        self.n_items = df[item_key].nunique()+1
        click_offsets = self.dataset.click_offsets
        session_idx_arr = self.dataset.session_idx_arr

        iters = np.arange(self.batch_size)
        maxiter = iters.max()
        start = click_offsets[session_idx_arr[iters]]
        end = click_offsets[session_idx_arr[iters] + 1]
        mask = [] # indicator for the sessions to be terminated
        finished = False        

        while not finished:
            minlen = (end - start).min()
            # Item indices (for embedding) for clicks where the first sessions start
            idx_target = df.item_idx.values[start]
            for i in range(minlen - 1):
                # Build inputs & targets
                idx_input = idx_target
                idx_target = df.item_idx.values[start + i + 1]
                input = idx_input
                target = idx_target
                yield input, target, mask
                
            # click indices where a particular session meets second-to-last element
            start = start + (minlen - 1)
            # see if how many sessions should terminate
            mask = np.arange(len(iters))[(end - start) <= 1]
            self.done_sessions_counter = len(mask)
            for idx in mask:
                maxiter += 1
                if maxiter >= len(click_offsets) - 1:
                    finished = True
                    break
                # update the next starting/ending point
                iters[idx] = maxiter
                start[idx] = click_offsets[session_idx_arr[maxiter]]
                end[idx] = click_offsets[session_idx_arr[maxiter] + 1]

In [6]:
def create_model():   
    emb_size = 50
    hidden_units = 100
    size = emb_size

    inputs = Input(batch_shape=(batch_size, 1, n_items))
    gru, gru_states = CuDNNGRU(hidden_units, stateful=True, return_state=True)(inputs)# drop1) #
    drop2 = Dropout(0.25)(gru)
    predictions = Dense(n_items, activation='softmax')(drop2)
    model = Model(input=inputs, output=[predictions])
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss=categorical_crossentropy, optimizer=opt)
    model.summary()

    filepath='./DwellTimeModel_checkpoint.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=2, save_best_only=True, mode='min')
    callbacks_list = []
    return model


def get_states(model):
    return [K.get_value(s) for s,_ in model.state_updates]


def set_states(model, states):
    for (d,_), s in zip(model.state_updates, states):
        K.set_value(d, s)


def get_recall(model, loader, epoch, train_generator_map, recall_k=20):

    test_dataset = SessionDataset(test_data, itemmap=train_generator_map)
    test_generator = SessionDataLoader(test_dataset, batch_size=batch_size)

    n = 0
    suma = 0
    suma_baseline = 0

    for feat, label, mask in test_generator:

        input_oh = to_categorical(feat, num_classes=loader.n_items) 
        input_oh = np.expand_dims(input_oh, axis=1)
        target_oh = to_categorical(label, num_classes=loader.n_items)
        pred = model.predict(input_oh, batch_size=batch_size)

        if n%100 == 0:
            try:
                print("{}:{}".format(n, suma/n))
            except:
                pass

        for row_idx in range(feat.shape[0]):
            pred_row = pred[row_idx] 
            label_row = target_oh[row_idx]

            idx1 = pred_row.argsort()[-recall_k:][::-1]
            idx2 = label_row.argsort()[-1:][::-1]

            n += 1
            if idx2[0] in idx1:
                suma += 1

    print("Recall@{} epoch {}: {}".format(recall_k, epoch, suma/n))


def get_mrr(model, loader,epoch,train_generator_map, mrr_k=20):

    test_dataset = SessionDataset(test_data, itemmap = train_generator_map)
    test_generator = SessionDataLoader(test_dataset, batch_size=batch_size)

    n = 0
    suma = 0
    suma_baseline = 0

    for feat, label, mask in test_generator:
        input_oh = to_categorical(feat, num_classes=loader.n_items) 
        input_oh = np.expand_dims(input_oh, axis=1)
        target_oh = to_categorical(label, num_classes=loader.n_items)
        pred = model.predict(input_oh, batch_size=batch_size)

        if n%100 == 0:
            try:
                print("{}:{}".format(n, suma/n))
            except:
                pass

        for row_idx in range(feat.shape[0]):
            pred_row = pred[row_idx] 
            label_row = target_oh[row_idx]

            idx1 = pred_row.argsort()[-mrr_k:][::-1]
            idx2 = label_row.argsort()[-1:][::-1]

            n += 1
            if idx2[0] in idx1:
                suma += 1/int((np.where(idx1 == idx2[0])[0]+1))        

    print("MRR@{} epoch {}: {}".format(mrr_k, epoch, suma/n))


def train_model(model, save_weights = False, path_to_weights = True):
    train_dataset = SessionDataset(train_data)

    model_to_train = model

    with tqdm(total=train_samples_qty) as pbar:
        for epoch in range(1, 10):
            if path_to_weights:
                loader = SessionDataLoader(train_dataset, batch_size=batch_size)
            for feat, target, mask in loader:

                input_oh = to_categorical(feat, num_classes=loader.n_items) 
                input_oh = np.expand_dims(input_oh, axis=1)

                target_oh = to_categorical(target, num_classes=loader.n_items)

                tr_loss = model_to_train.train_on_batch(input_oh, target_oh)

                real_mask = np.ones((batch_size, 1))
                for elt in mask:
                    real_mask[elt, :] = 0

                hidden_states = get_states(model_to_train)[0]

                hidden_states = np.multiply(real_mask, hidden_states)
                hidden_states = np.array(hidden_states, dtype=np.float32)
                model_to_train.layers[1].reset_states(hidden_states)

                pbar.set_description("Epoch {0}. Loss: {1:.5f}".format(epoch, tr_loss))
                pbar.update(loader.done_sessions_counter)

            # get metrics for epoch
            get_recall(model_to_train, loader, epoch,train_dataset.itemmap)
            get_mrr(model_to_train, loader,epoch, train_dataset.itemmap)

            # save model
            if save_weights:
                model_to_train.save('./DwellTimeEpoch{}.h5'.format(epoch))


In [ ]:
    PATH_TO_TRAIN = 'processed_augmented_train.csv'
    PATH_TO_DEV = 'processed_dev.csv'
    PATH_TO_TEST = 'processed_test.csv'
    train_data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId':np.int64})
    dev_data = pd.read_csv(PATH_TO_DEV, sep='\t', dtype={'ItemId':np.int64})
    test_data = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})
    
    batch_size = 512
    session_max_len = 100
    embeddingp=False

    n_items = len(train_data['ItemId'].unique())+1
    print("Unique training items:", n_items)

    dev_n_items = len(dev_data['ItemId'].unique())+1
    print("Unique dev items:", dev_n_items)

    test_n_items = len(test_data['ItemId'].unique())+1
    print("Unique testing items:", test_n_items)

    train_samples_qty = len(train_data['SessionId'].unique()) 
    print("Training sessions:", train_samples_qty)

    dev_samples_qty = len(dev_data['SessionId'].unique()) 
    print("Dev sessions:",dev_samples_qty)

    test_samples_qty = len(test_data['SessionId'].unique())
    print("Testing sessions:", test_samples_qty)
    
    train_fraction = 1 # (1 / fraction) most recent session quantity to consider
    dev_fraction = 1

    train_offset_step=train_samples_qty//batch_size
    dev_offset_step=dev_samples_qty//batch_size
    test_offset_step=test_samples_qty//batch_size
    aux = [0]
    aux.extend(list(train_data['ItemId'].unique()))
    itemids = np.array(aux)
    itemidmap = pd.Series(data=np.arange(n_items), index=itemids) 
    
    model = create_model()
    
    train_model(model)

Unique training items: 11619
Unique dev items: 10105
Unique testing items: 10366
Training sessions: 19853
Dev sessions: 5749
Testing sessions: 5271
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (512, 1, 11619)           0         
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       [(512, 100), (512, 100)]  3516300   
_________________________________________________________________
dropout_1 (Dropout)          (512, 100)                0         
_________________________________________________________________
dense_1 (Dense)              (512, 11619)              1173519   
Total params: 4,689,819
Trainable params: 4,689,819
Non-trainable param

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`
  # Remove the CWD from sys.path while we load stuff.
  0%|          | 0/19853 [00:00<?, ?it/s]

Instructions for updating:
Use tf.cast instead.


Epoch 1. Loss: 4.89693:  98%|█████████▊| 19415/19853 [03:03<00:04, 102.16it/s]

12800:0.10875


...

Epoch 2. Loss: 4.98684: : 38830it [10:20, 119.86it/s]

25600:0.0921484375
38400:0.09044270833333333
51200:0.09078125
64000:0.08984375
76800:0.08985677083333334
89600:0.08930803571428571
102400:0.08943359375
115200:0.08989583333333333
128000:0.0899921875
140800:0.09
153600:0.09006510416666667
Recall@20 epoch 2: 0.08999544902912622
12800:0.019545199556501042
25600:0.019205847463375603
38400:0.018361507141115403
51200:0.018449809049048114
64000:0.018209048366386704
76800:0.01825272998695049
89600:0.018197605684662072
102400:0.01831909759808445
115200:0.018418418650165767
128000:0.018471468926730163
140800:0.0185290895828098
153600:0.01857808712122415


Epoch 3. Loss: 7.26977: : 38830it [14:35,  8.90s/it] 

MRR@20 epoch 2: 0.01858347535047806


Epoch 3. Loss: 4.75798: : 41225it [14:52, 122.53it/s]